In [115]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
import re
from datetime import datetime, timedelta
import os
import oracledb
from sqlalchemy import create_engine, text
from flask import Flask, render_template
import db
import time
import numpy as np

title = 'data engineer'
location = "San Jose, CA" 
#location = "Chicago, Illinois, United States"
#location = "San Jose, California, United States"
#location = "United States" 
job_count = 0
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
#url = f'{base_url}?f_E={exp_level}&f_TPR={post_date}&keywords="{title}"&location={location}&start={job_count}'
tracking_all_job_ids = db.get_all_job_ids()




### FUNCTION DEFINITIONS

In [11]:
# Sends a request for one page of results to LinkedIn to get a list of job titles
def get_job_results_page(title, location, job_count) -> list:
    base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
    exp_level = "2"  # 2==entry level
    post_date = "r604800"  # r604800==Past week
    url = f'{base_url}?f_E={exp_level}&f_TPR={post_date}&keywords="{title}"&location={location}&start={job_count}'
    print("job page URL: ", url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    all_job_titles = soup.find_all("li")
    return all_job_titles

# Parses out the job title from one listing
def parse_job_id(job):
    return job.find("div", {"class":"base-card"}).get("data-entity-urn").split(":")[3]

# Get all job ids from one results page
def get_job_ids(title, location, job_count):
    job_ids = []
    all_job_titles = get_job_results_page(title, location, job_count)
    if not all_job_titles:  # if no jobs titles in the list
        print("No job titles found!")
        return None 

    all_job_ids = list(map(parse_job_id, all_job_titles))
    print("all job ids: ", all_job_ids)
    return all_job_ids
    
# Format strings to be in a consistent format 
def string_format(s: str) -> str:
    s = s.replace(" ", "_")     
    s = s.replace("'", "")
    s = s.replace('"', "")

    return s
    


    
def get_job_details_linkedin(job_ids):
    job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
    l_all_job_info = []

    for id in job_ids:
        d_job_info = {}
        d_job_info["job_id"] = int(id)
        d_job_info["job_search_term"] = title
        d_job_info["location_search_term"] = location
        job_desc_url = job_url.format(id)
        print(job_desc_url)
        res = requests.get(job_desc_url)
        soup = BeautifulSoup(res.text, "html.parser")

        # If the response code is 429, then we need to slow down scraping
        if res.status_code == 429:
            print("Got blocked LinkedIn...Slow down!!")
            time.sleep(120)
        else:
            # Putting in a 2 second delay for scraping
            time.sleep(2)

        
        # Get the company name
        d_job_info["company"] = soup.find("div", {"class":"top-card-layout__card"}).find("a").find("img").get("alt")

        # Get the location
        d_job_info["location"] = soup.find("div", {"class":"topcard__flavor-row"}).find("span", {"class":"topcard__flavor--bullet"}).text.strip()

        # Get the job title
        d_job_info["job_title"] = soup.find("h2", {"class":"top-card-layout__title"}).text.strip()

        # Get the full job description 
        d_job_info["job_description"] = soup.find("div", {"class":"show-more-less-html__markup"}).get_text(separator=u"\n")
        
        # Get years of experience!!! Keep checking this as it may be buggy
        d_job_info["experience"] = re.findall(r".*\D\d{1,2}\D.*years?", d_job_info["job_description"])
        d_job_info["experience"] = "\n".join(d_job_info["experience"])
        
        # Get the max years experience
        if d_job_info["experience"]:
            d_job_info['max_exp'] = max(re.findall(r'\d{1,2}', d_job_info["experience"]))
        

        # Get Seniority level, Employment type, Job function, Industries
        job_criteria_list = soup.find("ul", {"class":"description__job-criteria-list"}).find_all("li")
        for criteria in job_criteria_list:
            criteria = criteria.text.split("\n") # convert lines to a list
            criteria = [i.strip() for i in criteria if i.strip()] # remove lines with only white space
            criteria[0] = criteria[0].replace(" ", "_").lower()
            d_job_info.update({criteria[0]:criteria[1]})

        # Get job posting date
        posting_date = soup.find("span", {"class":"posted-time-ago__text"}).text.strip()
        posting_num = int(re.match(r'\d{1,2}',posting_date).group())
        if "minute" in posting_date:
            d_job_info["posting_date"] = datetime.today() - timedelta(minutes=posting_num)
        elif "hour" in posting_date:
            d_job_info["posting_date"] = datetime.today() - timedelta(hours=posting_num)
        elif "day" in posting_date:
            d_job_info["posting_date"] = datetime.today() - timedelta(days=posting_num)
        else:
            d_job_info["posting_date"] = ""
        
        
        # !!! Get Other useful info with AI !!!

        
        # Get URL 
        d_job_info["url"] = soup.find("a", {"class":"topcard__link"}).get("href")
        print(d_job_info["url"])

        # Append the job info (dict) to the list of job info
        l_all_job_info.append(d_job_info)

    # Convert list of all job info dicts to a dataframe
    df_all_job_info = pd.DataFrame(l_all_job_info)
    df_all_job_info


In [27]:

job_ids = []
job_count = 0
max_jobs_to_scrape = 300
# I can't get a job_count for the search query, 
# so going to scrape till it can NOT find any more job ids
# or I'm going to scrape upto 300 jobs or 12 pages
while job_count <= max_jobs_to_scrape:
    page_of_ids = get_job_ids(title, location, job_count)
    if page_of_ids:
        job_ids.extend(page_of_ids)
    else:
        break
    job_count += 25
    
print(job_ids)

job page URL:  https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_E=2&f_TPR=r604800&keywords="data engineer"&location=St Louis, Missouri, United States&start=0
all job ids:  ['3711635924', '3708952912', '3714572531']
job page URL:  https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_E=2&f_TPR=r604800&keywords="data engineer"&location=St Louis, Missouri, United States&start=25
No job titles found!
['3711635924', '3708952912', '3714572531']


### Filter out Job IDs already in the database

In [109]:
# Convert string job ids to ints and convert to numpy array
job_ids = np.array(list(map(int, job_ids)))
# Jobs ids that exist in the database
job_ids_for_db = job_ids[np.in1d(job_ids, tracking_all_job_ids)]
job_ids_for_db
# Filter out job ids already in the database
job_ids_for_linkedin = job_ids[~np.in1d(job_ids, tracking_all_job_ids)]
job_ids_for_linkedin




array([3711635924], dtype=int64)

In [114]:




get_job_details_db(job_ids_for_db)


SELECT * FROM tbl_jobs WHERE job_id in (3708952912, 3714572531)


,job_id,company,location,job_title,job_description,experience,seniority_level,employment_type,job_function,industries,posting_date,url,max_exp
0,3708952912,BDO USA,"St Louis, MO",Tax Digital Transformation & Innovation Senior...,\n\nJob Description\nJob Summary:\nThe Digital...,Bachelor's degree and six (6) or more years o...,Entry level,Full-time,Finance and Accounting/Auditing,Accounting,2023-09-09 04:00:51,https://www.linkedin.com/jobs/view/tax-digital...,None
1,3714572531,ClearanceJobs,"St Louis, MO",Data Scientist with Security Clearance,\n Job Number: R0179261 Data Scientist\...,The Opportunity: Ever-expanding technology lik...,Entry level,Part-time,Engineering and Information Technology,Defense and Space Manufacturing,2023-09-07 04:00:52,https://www.linkedin.com/jobs/view/data-scient...,None


In [111]:
type(job_ids_for_db)

numpy.ndarray

### Get all the job IDs that already exist in the database

In [89]:
cs = '''(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.us-sanjose-1.oraclecloud.com))(connect_data=(service_name=ga3e236c6957ba6_oltpdb_high.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))'''
user="appuser"
password=os.environ['ORACLE_PASSWORD_APPUSER']
engine = create_engine(
    f'oracle+oracledb://{user}:{password}@{cs}'
    )

with engine.connect() as conn:
    df_jobs_exists = pd.read_sql_query(f'SELECT job_id FROM tbl_jobs WHERE job_id IN ({",".join(job_ids)})', conn)
               
df_jobs_exists

,job_id
0,3708157029
1,3714774167


### Filter out all the IDs that exist in the database

In [90]:
df_all_job_info = df_all_job_info[~df_all_job_info['job_id'].isin(list(df_jobs_exists['job_id']))]
df_all_job_info

,job_id,company,location,job_title,job_description,experience,seniority_level,employment_type,job_function,industries,posting_date,url
2,3715560870,TechFetch.com - On Demand Tech Workforce hirin...,"San Jose, CA",Remote - Azure Data Engineer with Data Bricks,"\n ""ALL our jobs are US based and candi...","""ALL our jobs are US based and candida...",Entry level,Part-time,Information Technology,IT Services and IT Consulting,2023-09-09 04:39:38.138934,https://www.linkedin.com/jobs/view/remote-azur...
3,3708150943,"Tanisha Systems, Inc","San Jose, CA",Azure Data Engineer with Data Bricks,\n ob role: \nAzure Data Engineer with ...,Required Skills/Qualifications\n6+ years\nDesi...,Entry level,Contract,Information Technology,Human Resources Services,2023-09-08 04:39:38.722594,https://www.linkedin.com/jobs/view/azure-data-...
4,3712851766,SPECTRAFORCE,"Cupertino, CA",Video Data Collector,\n\nJob Title: Video Data Collector\nDuration:...,,Entry level,Contract,Art/Creative and Writing/Editing,Staffing and Recruiting,2023-09-09 04:39:39.023121,https://www.linkedin.com/jobs/view/video-data-...
5,3707550926,"TalentBurst, an Inc 5000 company","Cupertino, CA",Video Data Collector #: 23-15206,\n\nJob Description\n Title: Video Data Collec...,,Entry level,Contract,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-09 04:39:39.328467,https://www.linkedin.com/jobs/view/video-data-...
6,3708148376,Russell Tobin,"Cupertino, CA",Process Technician I,\n\nTitle: Video Data Collector\nJob Type: 6 m...,,Entry level,Contract,Management and Manufacturing,Staffing and Recruiting,2023-09-08 04:39:39.735684,https://www.linkedin.com/jobs/view/process-tec...
7,3712609923,SPECTRAFORCE,"Cupertino, CA",Video Data Collector,\n\nJob Title: Video Data Collector\nDuration:...,,Entry level,Full-time,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-07 04:39:40.045584,https://www.linkedin.com/jobs/view/video-data-...
8,3707532523,"TalentBurst, an Inc 5000 company","Cupertino, CA",Video Data Collector,\n\nTitle: Video Data Collector\n \n Location:...,,Entry level,Contract,Accounting/Auditing and Finance,Staffing and Recruiting,2023-09-08 04:39:40.350584,https://www.linkedin.com/jobs/view/video-data-...
9,3707507616,ICONMA,"Cupertino, CA",Process Technician I,\n\nProcess Technician I\n \n \n \n \n Loca...,,Entry level,Contract,Management and Manufacturing,Staffing and Recruiting,2023-09-08 04:39:40.707149,https://www.linkedin.com/jobs/view/process-tec...


### Insert job info (dataframe) into the database

In [91]:
df_all_job_info.to_sql('tbl_jobs', engine, 'appuser', if_exists='append', index=False, method=None)

-1

### Double Check database insert

In [109]:
with engine.connect() as conn:
    df_check_database = pd.read_sql_query(f'SELECT * FROM tbl_jobs WHERE job_id IN ({",".join(job_ids)})', conn)


df_check_database.to_html(f"templates/{string_format(title)}_{string_format(location)}.html")

In [126]:
title = "Data Engineer"
location = "St Louis, MO"


with engine.connect() as conn:
        df_results = pd.read_sql_query(
            f"SELECT * FROM tbl_jobs WHERE location like '%{location}%'", conn)
        
for value in df_results.items():
    print(value[1].values)

[3708952912 3714572531]
['BDO USA' 'ClearanceJobs']
['St Louis, MO' 'St Louis, MO']
['Tax Digital Transformation & Innovation Senior Data Engineer'
 'Data Scientist with Security Clearance']
['\n\nJob Description\nJob Summary:\nThe Digital Transformation & Innovation (DT&I) Data Manager will work with the Data Engineering and Integration team to help develop and support BDO’s Tax Data Warehouse, manage and maintain data ingestion and Extract/Transform/Load processes, as well as provide support for the Power BI business intelligence platform and enterprise applications supporting the tax practice.\nThe DT&I Data Manager will build and maintain tax data pipelines to support ad-hoc analytics and business intelligence applications as well as identify valuable development opportunities and ideas for improvement. This role will collaborate closely with data analytics teams to design, develop, and deploy new solutions that support strategic business priorities.\nJob Duties\n Develops software

In [160]:
cs = '''(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.us-sanjose-1.oraclecloud.com))(connect_data=(service_name=ga3e236c6957ba6_oltpdb_high.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))'''
user = "appuser"
password = os.environ['ORACLE_PASSWORD_APPUSER']
engine = create_engine(
    f'oracle+oracledb://{user}:{password}@{cs}'
)


with engine.begin() as conn:
        df_results = pd.read_sql_query(f"SELECT * FROM tbl_jobs", conn)
        #results = conn.execute(text(f"SELECT * FROM tbl_jobs WHERE job_title like '%{title}%'"))
        #results = conn.execute(text(f"SELECT * FROM tbl_jobs"))
        results = df_results.to_dict("records")


#jobs = db.query_db(title, location)

for job in results:
        print(job["job_id"])
        print(job["company"])
        print(job["location"])
        print(job["job_title"])
        print(job["job_description"])
        print(job["experience"])
        print(job["seniority_level"])
        print(job["employment_type"])
        print(job["job_function"])
        print(job["industries"])
        print(job["posting_date"])
        print(job["url"])



3708952912
BDO USA
St Louis, MO
Tax Digital Transformation & Innovation Senior Data Engineer


Job Description
Job Summary:
The Digital Transformation & Innovation (DT&I) Data Manager will work with the Data Engineering and Integration team to help develop and support BDO’s Tax Data Warehouse, manage and maintain data ingestion and Extract/Transform/Load processes, as well as provide support for the Power BI business intelligence platform and enterprise applications supporting the tax practice.
The DT&I Data Manager will build and maintain tax data pipelines to support ad-hoc analytics and business intelligence applications as well as identify valuable development opportunities and ideas for improvement. This role will collaborate closely with data analytics teams to design, develop, and deploy new solutions that support strategic business priorities.
Job Duties
 Develops software that processes, stores, and serves data for use by others 
 Develops large scale data structures and pipel

In [147]:
location

'San Jose, California, United States'

In [234]:
with engine.connect() as conn:
    tracking_all_job_ids = pd.read_sql_query(f'SELECT job_id FROM tbl_jobs', conn)
    tracking_all_job_ids = tracking_all_job_ids.to_numpy()

In [236]:
tracking_all_job_ids.to_numpy()

array([[3610297973],
       [3674224518],
       [3676035101],
       [3705364410],
       [3707507616],
       [3707532523],
       [3707550926],
       [3708148376],
       [3708150943],
       [3708157029],
       [3708952912],
       [3708953838],
       [3708954564],
       [3708958224],
       [3708984293],
       [3712609923],
       [3712851766],
       [3714572531],
       [3714774167],
       [3715560870]], dtype=int64)

In [20]:
start = 3708985762
np_test = np.arange(start, start+10000000) # 10 million
np_test

array([3708985762, 3708985763, 3708985764, ..., 3718985759, 3718985760,
       3718985761], dtype=int64)

array([3708952912, 3714572531], dtype=int64)

In [10]:
3714572531 in np_test

True

In [232]:
import sys

print(len(test_range))
print(sys.getsizeof(test_range[1])*len(test_range)/1024/1024, "MB")

print(len(np_test))
print(sys.getsizeof(np_test)/1024/1024, "MB")



1000000
30.517578125 MB
10000000
76.29405212402344 MB
